v0.01

In [2]:
import numpy as np
import os
import pandas as pd
import sys
import urllib.request
import warnings

import re
import time

from copy import copy, deepcopy

In [3]:
# !if [ ! -f "/content/helpers/parse_ud_features.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/parse_ud_features.py; fi


In [4]:
IS_GUEST = False
LOAD_SAVED_DATA = False # If False, running this notebook takes ~ 3 hours (Google Colab/CPU).

In [15]:
if IS_GUEST:
    root_dir = '.'
else:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"

# data_raw_dir = f"{root_dir}/data/raw"
# data_clean_dir = f"{root_dir}/data/clean"
data_processed_dir = f"{root_dir}/data/processed"
# helpers_dir = f"{root_dir}/helpers"

if root_dir not in sys.path:
    sys.path.append(root_dir)
    # sys.path.append(helpers_dir)

sentences_pos = f'{data_processed_dir}/sent_wikipedia_nlp_more_features.csv'

Mounted at /content/drive


In [16]:
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

###### Load csv with sentences with pos and morph tagging


In [17]:
df_sentences = pd.read_csv(sentences_pos)

In [18]:
df_sentences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787244 entries, 0 to 787243
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sentence  787244 non-null  object 
 1   words     787244 non-null  object 
 2   lemmas    787244 non-null  object 
 3   pos       787244 non-null  object 
 4   morph     787244 non-null  object 
 5   features  787244 non-null  object 
 6   dep       787244 non-null  object 
 7   n_words   787244 non-null  float64
 8   gender    787244 non-null  object 
 9   number    787244 non-null  object 
 10  person    787244 non-null  object 
dtypes: float64(1), object(10)
memory usage: 66.1+ MB


###### Add column for tracking error type
0 - N/A
1 - no error
2 - ...

In [19]:
df_sentences['error_type'] = 0

In [33]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
df_sentences[['sentence', 'morph', 'features']].head(10)

,sentence,morph,features
0,Става световен шампион в тежка категория през 2007 г.,"['Vniif-r3s', 'Amsi', 'Ncmsi', 'R', 'Afsi', 'Ncfsi', 'R', 'Mofsi', 'Ncfsi']","['Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing|NumType=Ord', 'Definite=Ind|Gender=Fem|Number=Sing']"
1,Лечението е трудно и продължително.,"['Ncnsd', 'Vxitf-r3s', 'Ansi', 'Cp', 'Ansi', 'punct']","['Definite=Def|Gender=Neut|Number=Sing', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_']"
2,"Въпросните тестови случаи се сформират на база спецификациите и изискванията, т.е. въз основа на това за какво е предназначена дадено приложение.","['A-pd', 'A-pi', 'Ncmpi', 'Ppxta', 'Vpitf-r3p', 'R', 'Ncfsi', 'Ncfpd', 'Cp', 'Ncnpd', 'punct', 'Cc', 'R', 'Ncfsi', 'R', 'Pde-os-n', 'R', 'Pia--s-n', 'Vxitf-r3s', 'Vpptcv--sfi', 'Ansi', 'Ncnsi', 'punct']","['Definite=Def|Degree=Pos|Number=Plur', 'Definite=Ind|Degree=Pos|Number=Plur', 'Definite=Ind|Gender=Masc|Number=Plur', 'Case=Acc|PronType=Prs|Reflex=Yes', 'Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', '_', 'Definite=Ind|Gender=Fem|Number=Sing', 'Definite=Def|Gender=Fem|Number=Plur', '_', 'Definite=Def|Gender=Neut|Number=Plur', '_', '_', '_', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Case=Nom|Gender=Neut|Number=Sing|PronType=Dem', '_', 'Gender=Neut|Number=Si..."
3,"През този сезон 2009 клубът носи името ""Гаосюн Яоти"".","['R', 'Pde-os-m', 'Ncmsi', 'Mc-pi', 'Ncmsf', 'Vpitf-r3s', 'Ncnsd', 'punct', 'Amsi', 'Ncmsi', 'punct', 'punct']","['_', 'Gender=Masc|Number=Sing|PronType=Dem', 'Definite=Ind|Gender=Masc|Number=Sing', 'Definite=Ind|Number=Plur|NumType=Card', 'Definite=Def|Gender=Masc|Number=Sing', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Def|Gender=Neut|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_', '_']"
4,От всички дъгови бои при колумбийските географското разпространение е най-северно.,"['R', 'Pce-op', 'A-pi', 'Ncmpi', 'R', 'A-pd', 'Ansd', 'Ncnsi', 'Vxitf-r3s', 'Ansi', 'punct']","['_', 'Case=Nom|Number=Plur|PronType=Tot', 'Definite=Ind|Degree=Pos|Number=Plur', 'Definite=Ind|Gender=Masc|Number=Plur', '_', 'Definite=Def|Degree=Pos|Number=Plur', 'Definite=Def|Degree=Pos|Gender=Neut|Number=Sing', 'Definite=Ind|Gender=Neut|Number=Sing', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_']"
5,"Официално днес носи името ""Свети Модест"" .","['Dm', 'Dt', 'Vpitf-r3s', 'Ncnsd', 'punct', 'Amsi', 'Npmsi', 'punct', 'punct']","['Degree=Pos', 'Degree=Pos', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Def|Gender=Neut|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_', '_']"
6,Достъпът до двигателя се осъществява през люк в десния борд.,"['Ncmsf', 'R', 'Ncmsh', 'Ppxta', 'Vpitf-r3s', 'R', 'Ncmsi', 'R', 'Amsh', 'Ncmsi', 'punct']","['Definite=Def|Gender=Masc|Number=Sing', '_', 'Definite=Def|Gender=Masc|Number=Sing', 'Case=Acc|PronType=Prs|Reflex=Yes', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', '_', 'Definite=Ind|Gender=Masc|Number=Sing', '_', 'Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_']"
7,Между българите има повече от 1 млн. мохамедани.,"['R', 'Ncmpd', 'Vnitf-r3s', 'Md-pi', 'R', 'Mcmsi', 'Ncmt', 'Ncmpi', 'punct']","['_', 'Definite=Def|Gender=Masc|Number=Plur', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'D

In [ ]:
# prompt: in jupyter notebook increate the horizontal size of the ouptput

from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))